In [2]:
import tensorflow as  tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot= True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1

k_conv1 = 5
k_conv2 = 5
k_maxpool1 = 2
k_maxpool2 = 2

n_hidden = 1024
n_out = 10

input_to_hidden_layer = (input_width//(k_maxpool1*k_maxpool2)) * (input_height//(k_maxpool1*k_maxpool2)) * n_conv2

In [34]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([k_conv1, k_conv1, input_channels, n_conv1])), 
    'wc2' : tf.Variable(tf.random_normal([k_conv2, k_conv2, n_conv1, n_conv2])),
    'wh1' : tf.Variable(tf.random_normal([input_to_hidden_layer , n_hidden])),
    'wout': tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([n_conv1])),
    'bc2' : tf.Variable(tf.random_normal([n_conv2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out]))
}

In [47]:
def conv(x, weight, bias, stride = 1):
    out = tf.nn.conv2d(x, weight, padding ="SAME", strides = [1, stride, stride, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k=4):
    return tf.nn.max_pool(x, padding="SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [74]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1 , input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, k_maxpool1)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, k_maxpool2)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_to_hidden_layer])
    hidden_out_bef_act = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_out_bef_drop = tf.nn.relu(hidden_out_bef_act)
    hidden_output = tf.nn.dropout(hidden_out_bef_drop, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights['wout']), biases['bo'])
    return output

In [75]:
x = tf.placeholder("float", (None, input_pixels))
y = tf.placeholder(tf.int32, (None, n_out))
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [69]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [70]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [71]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [72]:
batch_size=100
for iter in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c,opt = sess.run([cost, optimize], feed_dict={x:batch_x, y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

1055305.5976204872
31963.717243090272
17832.937511173343
13831.188849571943
10907.485426803236
11659.730582489185
10290.415113094106
10312.420976169407
7811.56733774266
8845.102045812328
7672.784175464883
6347.362620961666
5473.963523150771
6178.083966949848
4625.546682296991
3975.4329256783503
4168.47102065151
3879.654568880914
3115.6893429988254
2420.00682587416
3330.79317618148
2485.209906313494
1948.366869041221
2449.7567500682176
2100.4813107866576


In [76]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})
correct_preds.sum()

9854